<a href="https://colab.research.google.com/github/SARTHAK4U/GPU-Parallel-Image-Steganography/blob/master/OpenMpSteganAudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **----------------------------------------------------------------------------------------------------**

# **OPENMP BASED STEGANOGRAPHY** 

In [ ]:
	#Encryption

In [42]:
%%writefile parallel_encrypt.cpp
#include "/content/ppmHelper.h"
#include "/content/mp3Helper.h"
#include <immintrin.h>
#include <iostream>
#include <fstream>
#include <cmath>
#include <functional>
#include <chrono>
#include <cassert>
#include <random>
#include <omp.h>
const std::string pendl = ".\n";
#include <bits/stdc++.h>
using namespace std;


int main(int argc, char *argv[]){

    char *inputImageFile = "/content/msd.ppm";
    char *inputAudioFile = "/content/a.mp3";

    //dealing with image
     PPMimg *inpImg = readPPM(inputImageFile);
    int width = inpImg->width;
    int height = inpImg->height;
    long long totPixels = (long long)width * height;

    PPMpixel *inData = inpImg->data;
    PPMpixel *outData = (PPMpixel *)malloc(sizeof(PPMpixel) * totPixels);//size of one pixel * total pixels

    unsigned char * inputImageData = ppmTochar(inData, width, height); //converting arrays of pixels to linear array

    unsigned char * outputImageData = (unsigned char *)malloc(totPixels * 3ll);


     // Read input audio file
    MP3File *inpAudio = readMP3(inputAudioFile);
    char *audioData = inpAudio->data;


//------------------------------------------------------------
    cout << "Size of text file = " << inpAudio->size << " bytes ("
         << (inpAudio->size * 8) << " bits)\n";
    cout << "Size of image file = " << totPixels * 3 << " bytes\n";

    long long audioSize = inpAudio -> size; //total chunks of 1 byte read

  if((inpAudio->size * 8)> totPixels * 3ll) {
			std::cout << "ERROR: message too large to encode in input image." << pendl;
			exit(1);
		}

    int num=1,max=100;

    while(num<=max)
   {
       cout<<"NO. OF THREADS : "<<num<<endl;
		std::cout << "=== Beginning encoding ===" << std::endl;
	clock_t startTime = clock();

    #pragma omp parallel for num_threads(num) shared(inputImageData)
		for(size_t i = 0; i <audioSize; ++i) {
			//loop through each character in the message string
			for(size_t j = 0; j < 8; j++) {
				unsigned int val = audioData[i] >> (7 - j);
				val &= 1;
				size_t index = (8 * i) + j;
				unsigned int _data = inputImageData[index] & !(1);
				inputImageData[index] = (_data | val);
			}
		}

    	clock_t endTime = clock();
	
// Time calculation
  double Time = (double)(endTime - startTime) / CLOCKS_PER_SEC;
  cout << "Time taken (encrypt) = " << Time * 1000 << " ms\n";


		std::cout << "Message successfully encoded, writing to parallel_output.ppm"  << pendl;

    char outputImageFile[] = "/content/parallel_output.ppm";
    writePPM(outputImageFile, inputImageData, inpImg->width, inpImg->height, 3);
    num++;
}
}




Overwriting parallel_encrypt.cpp


In [43]:
%%shell
g++ parallel_encrypt.cpp

parallel_encrypt.cpp: In function ‘int main(int, char**)’:
parallel_encrypt.cpp:19:28: warning: ISO C++ forbids converting a string constant to ‘char*’ [-Wwrite-strings]
     char *inputImageFile = "/content/msd.ppm";
                            ^~~~~~~~~~~~~~~~~~
parallel_encrypt.cpp:20:28: warning: ISO C++ forbids converting a string constant to ‘char*’ [-Wwrite-strings]
     char *inputAudioFile = "/content/a.mp3";
                            ^~~~~~~~~~~~~~~~


In [ ]:
%%shell
./a.out

In [30]:
#Decryption

In [31]:
%%writefile parallel_decrypt.cpp
#include "/content/ppmHelper.h"
#include "/content/mp3Helper.h"
#include <immintrin.h>
#include <iostream>
#include <fstream>
#include <cmath>
#include <functional>
#include <chrono>
#include <cassert>
#include <random>
#include <omp.h>
const std::string pendl = ".\n";
#include <bits/stdc++.h>
using namespace std;
int main(int argc, char *argv[]){

char const *inputImageFile = "/content/parallel_output.ppm";
    long long audioSize = 117490;
    char *outputFileExtension = ".mp3";    
    
    
    // Read input image
    PPMimg *inpImg = readPPM(inputImageFile);
    int width = inpImg->width;
    int height = inpImg->height;
    long long totPixels = (long long)width * height;

    PPMpixel *inData = inpImg->data;
    PPMpixel *outData = (PPMpixel *)malloc(sizeof(PPMpixel) * totPixels);
    unsigned char *inputImageData = ppmTochar(inData, width, height);


    char *Data = (char *)malloc(audioSize);


    //parallel algorithm

clock_t startTime = clock();
  #pragma omp parallel for num_threads(omp_get_max_threads()) shared(inputImageData)
		for(size_t i = 0; i <audioSize; ++i) {
        size_t index = (8 * i);
        unsigned char Byte = 0;
        Byte |= (inputImageData[index  + 0] & 1) << 7;
        Byte |= (inputImageData[index  + 1] & 1) << 6;
        Byte |= (inputImageData[index  + 2] & 1) << 5;
        Byte |= (inputImageData[index  + 3] & 1) << 4;
        Byte |= (inputImageData[index  + 4] & 1) << 3;
        Byte |= (inputImageData[index  + 5] & 1) << 2;
        Byte |= (inputImageData[index  + 6] & 1) << 1;
        Byte |= (inputImageData[index  + 7] & 1) << 0;
        Data[i] = Byte;
    }

	clock_t endTime = clock();
	
      // Time calculation
  double Time = (double)(endTime - startTime) / CLOCKS_PER_SEC;
  cout << "Time taken (decrypt) = " << Time * 1000 << " ms\n";

    // Writing back audio file
    char outputAudioFile[] = "parallel_output";
    strcat(outputAudioFile, outputFileExtension);
    writeMP3(outputAudioFile, Data, audioSize);
    //--------------------------------------------------------------------------//
}

Overwriting parallel_decrypt.cpp


In [32]:
%%shell
g++ parallel_decrypt.cpp

parallel_decrypt.cpp: In function ‘int main(int, char**)’:
parallel_decrypt.cpp:19:33: warning: ISO C++ forbids converting a string constant to ‘char*’ [-Wwrite-strings]
     char *outputFileExtension = ".mp3";
                                 ^~~~~~


In [33]:
%%shell
./a.out

Time taken (decrypt) = 1.551 ms
